In [ ]:
%%capture
# Download dependcies
%pip install -q jmd_imagescraper
%pip install tensorflow_datasets

In [ ]:
%%capture
# Import Dependencies
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
from pathlib import Path
root = Path().cwd()/"images"

from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

duckduckgo_search(root, "Cats", "cute kittens", max_results=100)
duckduckgo_search(root, "Dogs", "cute puppies", max_results=100)